In [1]:
import os
import numpy as np
import cv2
import argparse
import imutils
import time
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

In [2]:
# Directory paths
TRAINING_DIR = "C:\\Users\\nisha\\Downloads\\Dataset\\train"
VALIDATION_DIR = "C:\\Users\\nisha\\Downloads\\Dataset\\test"

In [3]:
# Image dimensions and batch size
IMG_WIDTH, IMG_HEIGHT = 150, 150
BATCH_SIZE = 10

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
# Data normalization for validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='binary'  # Use 'binary' for binary classification (withmask and withoutmask)
)

Found 1315 images belonging to 2 classes.


In [5]:
# Validation data generator
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(224,224),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 194 images belonging to 2 classes.


In [6]:
# load the face mask detector model from disk
maskNet = load_model("C:\\Users\\nisha\\Projects\\mask_detector.model")

In [7]:
maskNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['Conv1_pad[0][0]']           
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                          

In [8]:
# load our serialized face detector model from disk
# Set the paths to the deploy.prototxt and the pre-trained Caffe model
prototxtPath = "C:\\Users\\nisha\\Downloads\\face-mask-detector-master\\face-mask-detector-master\\face_detector\\deploy.prototxt"
weightsPath = "C:\\Users\\nisha\\Downloads\\face-mask-detector-master\\face-mask-detector-master\\face_detector\\res10_300x300_ssd_iter_140000.caffemodel"

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [12]:
# initialize the video stream and allow the camera sensor to warm up
vs = VideoStream(src=0).start()
time.sleep(2.0)

In [13]:
def detectFaceMask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference, make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        preds = maskNet.predict(faces)

    # return a 2-tuple of the face locations and their corresponding
    # predictions
    return (locs, preds)

In [14]:
# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 800 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=800)

    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detectFaceMask(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        # Check if preds contain two probabilities (mask and withoutMask)
        if len(pred) == 2:
            (mask, withoutMask) = pred
        else:
            # Assume default probabilities for "mask" and "withoutMask"
            # Change these values based on your model output
            mask, withoutMask = 0.5, 0.5
        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Face Mask" if mask > withoutMask else "No Face Mask"

        color = (0, 255, 0) if label == "Face Mask" else (0, 0, 255)

        # include the probability in the label
        # label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the 'q' key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

1/1 [==============================] - 0s 34ms/step
